# REE Costing Framework Part 2 - Advanced Features

The purpose of this tutorial is to introduce advanced features available through the REE Costing Framework in PrOMMiS. To learn the basics of the REE Costing Framework, please refer to [Part 1](http://localhost:8888/notebooks/GitHub/PrOMMiS/prommis/docs/tutorials/ree_costing_framework_basic_features.ipynb) of this tutorial.

The tutorial will take users through the following:

* Importing and using costing from WaterTAP
* Importing capital costing from a custom method
* Setting up the costing block to calculate the net present value
* Adding cost bounds calculations to the model

Useful Links:
* Public GitHub Repository: https://github.com/prommis/prommis/tree/main
* REE Costing Module Code: https://github.com/prommis/prommis/blob/main/src/prommis/uky/costing/ree_plant_capcost.py

# 1 Import the necessary tools

First, import the required Python, Pyomo, IDAES, and PrOMMiS packages. These will be implemented at various stages of the tutorial.

In [1]:
# Pyomo packages
from pyomo.environ import (
    ConcreteModel,
    TransformationFactory,
    Constraint,
    Var,
    Param,
    units as pyunits,
    value,
)
from pyomo.network import Arc

# IDAES packages
from idaes.core import FlowsheetBlock, UnitModelBlock, UnitModelCostingBlock
from idaes.core.solvers import get_solver
from idaes.models.unit_models import Feed, Product


# PrOMMiS packages
from prommis.uky.costing.ree_plant_capcost import QGESSCosting, QGESSCostingData
from prommis.leaching.leach_train import LeachingTrain, LeachingTrainInitializer
from prommis.leaching.leach_reactions import CoalRefuseLeachingReactions
from prommis.leaching.leach_solids_properties import CoalRefuseParameters
from prommis.leaching.leach_solution_properties import LeachSolutionParameters
from prommis.leaching.leach_flowsheet import set_inputs, set_scaling

# WaterTAP packages
from prommis.nanofiltration.nf_brine import (
    define_feed_composition,
    initialize,
    solve_model,
    unfix_optimization_variables,
    add_objective,
    add_pressure_constraint,
)
from watertap.core.solvers import get_solver as get_watertap_solver  # WaterTAP has its own solver

from watertap.property_models.multicomp_aq_sol_prop_pack import (
    MCASParameterBlock,
)
from watertap.unit_models.nanofiltration_DSPMDE_0D import NanofiltrationDSPMDE0D
from watertap.unit_models.pressure_changer import Pump

# 2 Import and Use Costing From WaterTAP

The REE Costing Framework supports integration with costing methods imported from [WaterTAP](https://watertap.readthedocs.io/en/stable/), an external package for modeling water treatment operations such as aqueous membrane processes involving REE components. As demonstrated in [Part 1](http://localhost:8888/notebooks/GitHub/PrOMMiS/prommis/docs/tutorials/ree_costing_framework_basic_features.ipynb) of this tutorial, cost accounts imported from the built-in dictionary may be attached to either unit operation models or generic `UnitModelBlock` objects. In contrast, WaterTAP costing methods look for the unit model class, i.e. to import costing for a WaterTAP model the flowsheet needs to contain that WaterTAP model before calling `build_process_costs`. The heavy lifting for passing along unit model references, importing costing methods, and extracting the necessary capital and operating cost components occurs inside the REE Costing Framework. On the flowsheet side, users only need to define their unit models and pass them to the costing method.

## 2.1 Build a flowsheet to cost

For this example, we will build a flowsheet containing one PrOMMiS model, one WaterTAP model, and one `UnitModelBlock` representing a supplemental unit to be costed. We'll see that the the `build_process_costs` method will automatically extract the relevant costing from all three models.

To begin, let's create the initial flowsheet:

In [2]:
# Create a Concrete Model as the top level object
m = ConcreteModel()

# Add a flowsheet object to the model
m.fs = FlowsheetBlock(dynamic=False)

# attach a flowsheet costing block
m.fs.costing = QGESSCosting()

## 2.2 Add PrOMMiS unit model

For this tutorial, the PrOMMiS model is the `LeachingTrain`:

In [3]:
# build the LeachingTrain properties
m.fs.leach_soln = LeachSolutionParameters()
m.fs.coal = CoalRefuseParameters()
m.fs.leach_rxns = CoalRefuseLeachingReactions()

# build the LeachingTrain model
m.fs.leach = LeachingTrain(
    number_of_tanks=1,
    liquid_phase={
        "property_package": m.fs.leach_soln,
        "has_energy_balance": False,
        "has_pressure_balance": False,
    },
    solid_phase={
        "property_package": m.fs.coal,
        "has_energy_balance": False,
        "has_pressure_balance": False,
    },
    reaction_package=m.fs.leach_rxns,
)

# set input specifications
set_inputs(m)

# set scaling for the model
set_scaling(m)

# Create a scaled version of the model to solve
scaling = TransformationFactory("core.scale_model")
scaled_model = scaling.create_using(m, rename=False)

# Initialize model
# This is likely to fail to converge, but gives a good enough starting point
initializer = LeachingTrainInitializer()
try:
    initializer.initialize(scaled_model.fs.leach)
except:
    pass

# Solve scaled model
solver = get_solver()
solver.solve(scaled_model, tee=True)

# Propagate results back to unscaled model
scaling.propagate_solution(scaled_model, m)

2024-07-30 10:39:52 [INFO] idaes.init.fs.leach.mscontactor: Stream Initialization Completed.
2024-07-30 10:39:52 [INFO] idaes.init.fs.leach.mscontactor: Initialization Completed, optimal - <undefined>
Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computatio

## 2.3 Add WaterTAP unit model

The WaterTAP model will be the `NanofiltrationDSPMDE0D` unit model, a time-dependent model implementing the Donnan Steric Pore Model with Dielectric Exclusion (DSPM-DE) for nanofiltration. More information on this model may be found in the [NanofiltrationDSPMDE0D documentation](https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/nanofiltration_dspmde_0D.html). The specific model is the [Nanofiltration Brine](https://github.com/prommis/prommis/tree/main/src/prommis/nanofiltration) model, which exists in the ProMMiS repository:

In [4]:
watertap_solver = get_watertap_solver()

# define the property model
default = define_feed_composition()
m.fs.properties = MCASParameterBlock(**default)

# add the feed and product streams
m.fs.feed = Feed(property_package=m.fs.properties)
m.fs.permeate = Product(property_package=m.fs.properties)
m.fs.retentate = Product(property_package=m.fs.properties)

# define unit models; note that the NF needs to named "unit" for the imported methods to work
m.fs.pump = Pump(property_package=m.fs.properties)
m.fs.unit = NanofiltrationDSPMDE0D(property_package=m.fs.properties)

# connect the streams and blocks
m.fs.feed_to_pump = Arc(source=m.fs.feed.outlet, destination=m.fs.pump.inlet)
m.fs.pump_to_nf = Arc(source=m.fs.pump.outlet, destination=m.fs.unit.inlet)
m.fs.nf_to_permeate = Arc(source=m.fs.unit.permeate, destination=m.fs.permeate.inlet)
m.fs.nf_to_retentate = Arc(source=m.fs.unit.retentate, destination=m.fs.retentate.inlet)
TransformationFactory("network.expand_arcs").apply_to(m)

# initialize
initialize(m, watertap_solver)

# solve
solve_model(m, watertap_solver)

# create optimization problem
unfix_optimization_variables(m)
add_objective(m)
add_pressure_constraint(m)

# re-solve
solve_model(m, watertap_solver)

('Liq', 'H2O') flow_mol_phase_comp scaling factor = 0.1
('Liq', 'Li_+') flow_mol_phase_comp scaling factor = 10
('Liq', 'Mg_2+') flow_mol_phase_comp scaling factor = 10
('Liq', 'Cl_-') flow_mol_phase_comp scaling factor = 1
Cl_- adjusted: fs.feed.properties[0.0].flow_mol_phase_comp['Liq',Cl_-] was adjusted from 4.843574775634025 and fixed to 0.9219738584555871. Electroneutrality satisfied for fs.feed.properties[0.0]. Balance Result = 1.1368683772161603e-13
2024-07-30 10:39:54 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.area
2024-07-30 10:39:54 [INFO] idaes.init.fs.feed: Initialization Complete.
2024-07-30 10:39:54 [INFO] idaes.init.fs.pump.control_volume: Initialization Complete
2024-07-30 10:39:54 [INFO] idaes.init.fs.pump: Initialization Complete: optimal - Optimal Solution Found
2024-07-30 10:39:56 [INFO] idaes.init.fs.unit: Initialization Complete: optimal - Optimal Solution Found
2024-07-30 10:39:56 [INFO] idaes.init.fs.permeate: Initialization Complete.


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 543, 'Number of variables': 544, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.1036372184753418}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Let's explore the unit we just added to check for costing objects; we'll also check for "io_list" to verify that the search is doing something:

In [5]:
for i in m.fs.unit.component_data_objects(descend_into=True):
    if "cost" in str(i) or "io_list" in str(i):
        print(i)

fs.unit.io_list


As expected, there are no costing objects on the model yet. We don't need to add this ourselves, as the REE Costing Framework will call the correct objects from WaterTAP automatically when we build the plant costs.

Finally, let's create a `UnitModelBlock` for our third unit:

In [6]:
m.fs.make_up_pumps = UnitModelBlock()

## 2.4 Build costing

The PrOMMiS model and `UnitModelBlock` correspond to named cost accounts, and each need their own `UnitModelCostingBlock`. We'll do this as follows:

In [7]:
# add costing for leach tanks - costing is attached to LeachingTrain model
m.fs.leach.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,  # this is the flowsheet costing block
    costing_method=QGESSCostingData.get_REE_costing,  # REE capital costing method
    costing_method_arguments={
        "cost_accounts": ["4.2",],  # leach tank account
        "scaled_param": m.fs.leach.volume[0, 1],
        "source": 1,
        "n_equip": 3,
        "scale_down_parallel_equip": False,
        "CE_index_year": "2021",
    },
)

# add costing for makeup pumps - costing is attached to UnitModelBlock
# the scaling parameter itself exists on the main leach model
# we needed the UnitModelBlock because a model can only have one UnitModelCostingBlock
m.fs.make_up_pumps.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=QGESSCostingData.get_REE_costing,
    costing_method_arguments={
        "cost_accounts": ["4.4",],
        "scaled_param": m.fs.leach.liquid_inlet.flow_vol[0],
        "source": 1,
        "n_equip": 3,
        "scale_down_parallel_equip": False,
        "CE_index_year": "2021",
    },
)

for i in m.fs.costing._registered_unit_costing:
    print(i.name)

fs.leach.costing
fs.make_up_pumps.costing


Now, let's build the plant costs with and without the WaterTap model. We see from the output above that the WaterTAP model is not currently in the registered costing. To let the costing framework know about the WaterTAP model, we need to pass an argument `watertap_blocks` as a list of unit models that required WaterTAP costing. We'll added fixed and variable operating costs so we can see how the WaterTAP option changes the model.

First, without the WaterTAP block:

In [8]:
# define product rates
pure_product_output_rates = {
    "Sc2O3": 1.9 * pyunits.kg / pyunits.hr,
    "Dy2O3": 0.4 * pyunits.kg / pyunits.hr,
    "Gd2O3": 0.5 * pyunits.kg / pyunits.hr,
}

mixed_product_output_rates = {
    "Sc2O3": 0.00143 * pyunits.kg / pyunits.hr,
    "Y2O3": 0.05418 * pyunits.kg / pyunits.hr,
    "La2O3": 0.13770 * pyunits.kg / pyunits.hr,
    "CeO2": 0.37383 * pyunits.kg / pyunits.hr,
    "Pr6O11": 0.03941 * pyunits.kg / pyunits.hr,
    "Nd2O3": 0.17289 * pyunits.kg / pyunits.hr,
    "Sm2O3": 0.02358 * pyunits.kg / pyunits.hr,
    "Eu2O3": 0.00199 * pyunits.kg / pyunits.hr,
    "Tb4O7": 0.00801 * pyunits.kg / pyunits.hr,
    "Tm2O3": 0.00130 * pyunits.kg / pyunits.hr,
    "Yb2O3": 0.00373 * pyunits.kg / pyunits.hr,
    "Lu2O3": 0.00105 * pyunits.kg / pyunits.hr,
}

# define resources
m.fs.water = Var([0], initialize=1000, units=pyunits.gallon / pyunits.hr)
m.fs.water.fix()

m.fs.chemicals = Var([0], initialize=20, units=pyunits.gallon / pyunits.hr)
m.fs.chemicals.fix()

# make a copy so that we can build the costs again later - users should not called build_process_costs
# on the same costing block twice, and it will throw an error if attempted
import copy
temp_m = copy.deepcopy(m)

# build process costs
temp_m.fs.costing.build_process_costs(
    Lang_factor=2.97,
    fixed_OM=True,
    pure_product_output_rates=pure_product_output_rates,
    mixed_product_output_rates=mixed_product_output_rates,
    variable_OM=True,
    resources=["water", "chemicals",],
    rates=[temp_m.fs.water, temp_m.fs.chemicals],
    prices={"chemicals": 1.00 * pyunits.USD_2021 / pyunits.gallon},
)

# solve and display results
solver.solve(temp_m, tee=False)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 566, 'Number of variables': 567, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.11460328102111816}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Checking some of the model objects:

In [9]:
for i in temp_m.fs.costing.component_data_objects(Var, descend_into=True):
    if "watertap" in str(i) or "BEC" in str(i):
        print(i.name, "has a value of ", value(i))

print()

for i in temp_m.fs.costing.component_data_objects(Constraint, descend_into=True):
    if "watertap" in str(i.expr) or "BEC" in str(i.expr):
        print(i.expr)
        print()

fs.costing.total_BEC has a value of  0.017224164457821052
fs.costing.watertap_fixed_costs has a value of  8.156630584998156e-56
fs.costing.watertap_variable_costs has a value of  0.0

fs.costing.total_BEC  ==  fs.leach.costing.bare_erected_cost['4.2'] + fs.make_up_pumps.costing.bare_erected_cost['4.4']

fs.costing.total_installation_cost  ==  (fs.costing.Lang_factor - 1)*fs.costing.total_BEC

fs.costing.total_plant_cost  ==  fs.costing.total_BEC + fs.costing.total_installation_cost + fs.costing.other_plant_costs

fs.costing.watertap_fixed_costs  ==  0

fs.costing.total_fixed_OM_cost  ==  fs.costing.annual_labor_cost + fs.costing.maintenance_and_material_cost + fs.costing.quality_assurance_and_control_cost + fs.costing.admin_and_support_labor_cost + fs.costing.sales_patenting_and_research_cost + fs.costing.property_taxes_and_insurance_cost + fs.costing.other_fixed_costs + fs.costing.watertap_fixed_costs + fs.costing.custom_fixed_costs

fs.costing.watertap_variable_costs  ==  0

fs.costi

We see that the model has variables called `watertap_fixed_costs` and `watertap_variable_costs`with values that are essentially zero. These are placeholder variables to catch operating costs returned by WaterTAP. The expression for `total_BEC` only contains the leach and makeup pump costs. There are constraints enforcing that the WaterTAP operating cost variables are equal to zero, which makes sense as no blocks were passed that required WaterTAP costing. Note that the variables are present in the fixed and variable O&M cost expressions.

Now, let's build the costs and pass the WaterTAP model:

In [10]:
# build process costs
m.fs.costing.build_process_costs(
    Lang_factor=2.97,
    fixed_OM=True,
    pure_product_output_rates=pure_product_output_rates,
    mixed_product_output_rates=mixed_product_output_rates,
    variable_OM=True,
    resources=["water", "chemicals",],
    rates=[m.fs.water, m.fs.chemicals],
    prices={"chemicals": 1.00 * pyunits.USD_2021 / pyunits.gallon},
    watertap_blocks=[m.fs.unit,],
)

# solve and display results
solver.solve(m, tee=False)

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 568, 'Number of variables': 569, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.11111021041870117}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Checking the same model objects as before:

In [11]:
for i in m.fs.costing.component_data_objects(Var, descend_into=True):
    if "watertap" in str(i) or "BEC" in str(i):
        print(i.name, "has a value of ", value(i))

print()

for i in m.fs.costing.component_data_objects(Constraint, descend_into=True):
    if "watertap" in str(i.expr) or "BEC" in str(i.expr):
        print(i.expr)
        print()

fs.costing.total_BEC has a value of  0.052441000096515866
fs.costing.watertap_fixed_costs has a value of  0.0035216835638694813
fs.costing.watertap_variable_costs has a value of  0.0

fs.costing.total_BEC  ==  fs.leach.costing.bare_erected_cost['4.2'] + fs.make_up_pumps.costing.bare_erected_cost['4.4'] + 1.1739346708671863e-06*(MUSD_2021/USD_2018)*fs.unit.costing.capital_cost

fs.costing.total_installation_cost  ==  (fs.costing.Lang_factor - 1)*fs.costing.total_BEC

fs.costing.total_plant_cost  ==  fs.costing.total_BEC + fs.costing.total_installation_cost + fs.costing.other_plant_costs

fs.costing.watertap_fixed_costs  ==  1.1739346708671863e-06*(MUSD_2021/USD_2018)*a*fs.unit.costing.fixed_operating_cost

fs.costing.total_fixed_OM_cost  ==  fs.costing.annual_labor_cost + fs.costing.maintenance_and_material_cost + fs.costing.quality_assurance_and_control_cost + fs.costing.admin_and_support_labor_cost + fs.costing.sales_patenting_and_research_cost + fs.costing.property_taxes_and_insuranc

The `NanofiltrationDSPMDE0D` costing only adds a fixed operating cost expression, and not a variable operating cost expression, so we'd expect the model to reflect that. As expected, `watertap_fixed_costs` has a nonzero value while `watertap_variable_costs` is still zero. The expressions for `total_BEC` and `watertap_fixed_costs` now each include a term for the `NanofiltrationDSPMDE0D` model.

Multiple WaterTAP models may be passed to the argument `watertap_blocks`, and the REE Costing Framework will automatically extract the costing variables and expressions.

# 3 Import and Use Custom Capital Costing

The `UnitModelCostingBlock` takes several arguments, as shown in the block used earlier to build the leach costing:

```
m.fs.leach.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,  # this is the flowsheet costing block
    costing_method=QGESSCostingData.get_REE_costing,  # REE capital costing method
    costing_method_arguments={
        "cost_accounts": ["4.2",],  # leach tank account
        "scaled_param": m.fs.leach.volume[0, 1],
        "source": 1,
        "n_equip": 3,
        "scale_down_parallel_equip": False,
        "CE_index_year": "2021",
    },
)
```

Inspecting the arguments, `costing_method` points to an external class containing a function that defines the cost model, and `costing_method_arguments` aligns with the inputs of that function. We can use our own custom class and function for `costing_method`.

## 3.1 Writing a Custom Cost Model

Suppose we have written our own costing model for a custom vessel. The capital cost (in dollars) assumes a power law based on vessel volume:
```
Capital_Cost = Reference_Cost * (Volume / Reference_Volume) ** 0.6
```

Fixed operating costs (in dollars) are a set percentage of the capital cost:
```
Fixed_Costs = 0.05 * Capital_Cost
```

Variable operating costs (in dollars per year) are a function of the water injection rate:
```
Variable_Operating_Costs = ($.00190/gal) * Water_Injection_Rate
```

We'll add arguments for the vessel volume, material, water injection rate, and number of units. We'll also allow a string for the cost year, which will set the units for the cost variables.

We formulate the model as follows:

In [12]:
from idaes.core import declare_process_block_class, FlowsheetCostingBlockData
from pyomo.environ import NonNegativeReals

@declare_process_block_class("CustomCosting")
class CustomCostingData(FlowsheetCostingBlockData):
    # Register custom currency units used in the custom costing model
    pyunits.load_definitions_from_strings(["USD_custom = 500/700 * USD_CE500",])

    # define global parameters - base cost year and period
    def build_global_params(self):

        # Set the base year for all costs
        self.base_currency = pyunits.USD_2021
        # Set a base period for all operating costs
        self.base_period = pyunits.year

    # custom costing method
    def cost_custom_vessel(
        blk,  # when the costing block is built, blk will be the costing block itself
        volume_per_unit=1000 * pyunits.m**3,
        material="carbonsteel",  #
        water_injection_rate_per_unit=1 * pyunits.m**3/pyunits.s,
        number_of_units=1,
        CE_index_year="2021",
    ):

        # alias for cost year units
        CE_index_units = getattr(pyunits, "MUSD_" + CE_index_year)

        # make parameter for number of units
        blk.number_of_units = Param(initialize=number_of_units, mutable=False)

        # define the bare erected cost per unit
    
        material_factor_dict = {  # material factors for each valid choice of shell material
            "carbonsteel": 1,
            "stainlessteel": 1.5,
            "aluminum": 2,
        }
    
        blk.capital_cost_per_unit = Var(
            initialize=1000,
            units=CE_index_units,
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        @blk.Constraint()
        def capital_cost_per_unit_eq(blk):
            # cost equation CAPITAL_COST = REF_COST * (VOLUME / REF_VOLUME)**0.6
            ref_cost = 10000 * pyunits.USD_custom
            ref_volume = 1000 * pyunits.m**3
            return (
                blk.capital_cost_per_unit == pyunits.convert(
                    material_factor_dict[material] * ref_cost* (volume_per_unit/ref_volume)**0.6,
                    to_units = CE_index_units
                )
            )
    
        # create a variable capital_cost that the REE Costing Framework can look for
        blk.capital_cost = Var(
            initialize=1000,
            units=CE_index_units,
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        @blk.Constraint()
        def capital_cost_constraint(blk):
            return blk.capital_cost == blk.capital_cost_per_unit * blk.number_of_units
    
        # define the fixed costs
    
        blk.fixed_operating_cost_per_unit = Var(
            initialize=1000,
            units=CE_index_units,
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        @blk.Constraint()
        def fixed_operating_cost_per_unit_eq(blk):
            return (
                blk.fixed_operating_cost_per_unit == pyunits.convert(
                    0.05 * blk.capital_cost,
                    to_units = CE_index_units
                )
            )
    
        # create a variable fixed_operating_cost that the REE Costing Framework can look for
        blk.fixed_operating_cost = Var(
            initialize=1000,
            units=CE_index_units,
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        @blk.Constraint()
        def fixed_operating_cost_constraint(blk):
            return blk.fixed_operating_cost == blk.fixed_operating_cost_per_unit * blk.number_of_units
    
        # define the variable costs
    
        blk.variable_operating_cost_per_unit = Var(
            initialize=1000,
            units=CE_index_units/pyunits.year,
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        @blk.Constraint()
        def variable_operating_cost_per_unit_eq(blk):
            return (
                blk.variable_operating_cost_per_unit == pyunits.convert(
                    0.00190 * pyunits.USD_custom/pyunits.gal * water_injection_rate_per_unit,
                    to_units = CE_index_units/pyunits.year
                )
            )
    
        # create a variable variable_operating_cost that the REE Costing Framework can look for
        blk.variable_operating_cost = Var(
            initialize=1000,
            units=CE_index_units/pyunits.year,
            domain=NonNegativeReals,
            bounds=(0, None),
        )

        @blk.Constraint()
        def variable_operating_cost_constraint(blk):
            return blk.variable_operating_cost == blk.variable_operating_cost_per_unit * blk.number_of_units

There are few important details in the model above. First, the decorator `@declare_process_block_class()` tells the Python interpreter to register the new class as an importable object with the name "CustomCosting". The class inherits all properties of the `FlowsheetCostingBlock` class by calling its corresponding data class.

The first line inside the custom class defines a new cost unit `USD_custom` with a Chemical Engineering Plant Cost Index (CEPCI) value of 700. `USD_CE500` is a reference unit container enabling conversion between currency units. The method `build_global_params` defines the base cost year and period for the model. Since the class inherits all properties of the `FlowsheetCostingBlockData` class, the final model will return cost variables in units of `USD_2021` and any time units will be in `year` or `a` (annum, the two are equivalent in Pyomo unit container syntax).

Then, the method `cost_custom_vessel` defines all cost equations. Each of the three cost variables (capital, fixed operating, variable operating) is built by defining a cost per unit with appropriate inputs, and then a total cost based on the number of units. The naming convention is important, since the REE Costing Framework will look for variables named `capital_cost`, `fixed_operating_cost`, and `variable_operating_cost` when building the total plant costs.

## 3.2 Create a Costing Block

Let's create a new flowsheet using our new costing. Since it's a custom model, we'll use a `UnitModelBlock` to add our own variables:

In [13]:
# Create a Concrete Model as the top level object
m = ConcreteModel()

# Add a flowsheet object to the model
m.fs = FlowsheetBlock(dynamic=False)

# attach a flowsheet costing block
m.fs.costing = QGESSCosting()

# create model
m.fs.custom_vessel = UnitModelBlock()
m.fs.custom_vessel.volume = Var(initialize=5000, units=pyunits.m**3)
m.fs.custom_vessel.volume.fix()
m.fs.custom_vessel.water_injection_rate = Var(initialize=0.1, units=pyunits.m**3/pyunits.s)
m.fs.custom_vessel.water_injection_rate.fix()

# add costing
m.fs.custom_vessel.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=CustomCostingData.cost_custom_vessel,
    costing_method_arguments={
        "volume_per_unit": m.fs.custom_vessel.volume,
        "material": "carbonsteel",
        "water_injection_rate_per_unit": m.fs.custom_vessel.water_injection_rate,
        "number_of_units": 3,
        "CE_index_year": "2022",
    },
)

m.fs.custom_vessel.display()

Block fs.custom_vessel

  Variables:
    volume : Size=1, Index=None, Units=m**3
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  5000 :  None :  True : False :  Reals
    water_injection_rate : Size=1, Index=None, Units=m**3/s
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.1 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None

  Blocks:
    Block fs.custom_vessel.costing
    
      Variables:
        capital_cost_per_unit : Size=1, Index=None, Units=MUSD_2022
            Key  : Lower : Value : Upper : Fixed : Stale : Domain
            None :     0 :  1000 :  None : False : False : NonNegativeReals
        capital_cost : Size=1, Index=None, Units=MUSD_2022
            Key  : Lower : Value : Upper : Fixed : Stale : Domain
            None :     0 :  1000 :  None : False : False : NonNegativeReals
        fixed_operating_cost_per_unit : Size=1, Index=None, Units=MUSD_2022
       

The costing block contains the custom model, and cost variables are in `MUSD_2022` as specified in the costing block definition. Note that the model is not solved yet, so the variables are at their default values.

## 3.3 Build Plant Costing

Now, we can build the plant costs. Like the prior example using WaterTAP, the extra variables and constraints will be added to the model as additional costs. However, because we defined the capital costing using a `UnitModelCostingBlock`, we see that the custom model is present in the registered cost for the flowsheet:

In [14]:
for block in m.fs.costing._registered_unit_costing:
    print(block.name)

fs.custom_vessel.costing


This means that we don't need to pass a special list including the unit. The REE Costing Framework will see that our custom model is already registered and will automatically check for `capital_cost`, `fixed_operating_cost`, and `variable_operating_cost` variables.

Note that we still need the product rates to calculate fixed operating costs. 

Let's build the costs using mainly default values to keep the code short:

In [15]:
# build process costs
m.fs.costing.build_process_costs(
    Lang_factor=2.97,
    fixed_OM=True,
    pure_product_output_rates=pure_product_output_rates,  # defined earlier, need for fixed O&M costs
    mixed_product_output_rates=mixed_product_output_rates,  # defined earlier, need for fixed O&M costs
    variable_OM=True,
    resources=[],  # need for variable O&M costs, empty list because we have no additional resources
    rates=[],  # need for variable O&M costs, empty list because we have no additional resources
)

# solve and display results
solver.solve(m, tee=False)

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 25, 'Number of variables': 25, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.07619524002075195}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [16]:
for i in m.fs.costing.component_data_objects(Var, descend_into=True):
    if "custom" in str(i):
        print(i.name, "has a value of ", value(i))

print()

for i in m.fs.costing.component_data_objects(Constraint, descend_into=True):
    if "custom" in str(i.expr):
        print(i.expr)
        print()

fs.costing.custom_fixed_costs has a value of  0.011954453692614863
fs.costing.custom_variable_costs has a value of  4.806189725449736

fs.costing.total_BEC  ==  0.8676470588235295*(MUSD_2021/MUSD_2022)*fs.custom_vessel.costing.capital_cost

fs.costing.custom_fixed_costs  ==  0.8676470588235295*(MUSD_2021/MUSD_2022)*fs.custom_vessel.costing.fixed_operating_cost

fs.costing.total_fixed_OM_cost  ==  fs.costing.annual_labor_cost + fs.costing.maintenance_and_material_cost + fs.costing.quality_assurance_and_control_cost + fs.costing.admin_and_support_labor_cost + fs.costing.sales_patenting_and_research_cost + fs.costing.property_taxes_and_insurance_cost + fs.costing.other_fixed_costs + fs.costing.watertap_fixed_costs + fs.costing.custom_fixed_costs

fs.costing.custom_variable_costs  ==  0.8676470588235297*(MUSD_2021/MUSD_2022)*fs.custom_vessel.costing.variable_operating_cost

fs.costing.total_variable_OM_cost[0.0]  ==  fs.costing.other_variable_costs[0.0] + fs.costing.plant_overhead_cost[0.0

Although we didn't change anything about the `build_process_costs` call to note the custom model, the costing framework extracted the cost variables from the custom vessel. The `total_BEC` expression is the capital cost of our custom vessel, since that is the only unit in the flowsheet, and we can see that it accounts for the unit conversion between the model units of `MUSD_2022` and the flowsheet units of `MUSD_2021`. The operating cost equations also include the correct variables from our custom model, with the proper unit conversions.

# 4 Calculate Net Present Value

If all required capital, fixed operating, and variable operating costs are calculated or provided, the REE Costing Framework supports estimation of the net present value (NPV). The NPV represents the total value of the plant construction, operation, and production over the plant lifetime taking into account the changing worth of money over time. For example, distributed capital costs over multiple years or operating costs per year over the entire lifetime of a plant may grow with an escalation or inflation rate. The NPV provides a way to normalize costs over time to the current dollar year. Typical NPV components include capital cost, operating cost, revenue (this is a positive contribution); additionally, royalties from product sales and depreciation that reduces equipment resale value at the end of the plant lifetime are included in the NPV calculation.

To calculate the NPV as part of the `build_process_costs` method, certain configuration arguments must be set when the flowsheet costing block is instantiated. The NPV calculation method will assume both fixed and variable operating costs exist, as they are required to calculate the NPV. Users may provide their own values instead to bypass this requirement (see Section 4.2).

# 4.1 Calculate NPV From Costing Block

To begin, we need to create our flowsheet costing block with several configuration arguments. Two arguments are required, the discount percentage defining cost projections over time and the plant lifetime in years. Many of the arguments have default values that are used if no other value is specified:

In [17]:
# Create a Concrete Model as the top level object
m = ConcreteModel()

# Add a flowsheet object to the model
m.fs = FlowsheetBlock(dynamic=False)

# attach a flowsheet costing block with configuration arguments for NPV calculation
m.fs.costing = QGESSCosting(
    # arguments for NPV
    discount_percentage=10,  # percent, required argument
    plant_lifetime=20,  # years, required argument
    has_capital_expenditure_period=True,  # whether capital occurs before the operating period, default False
    capital_expenditure_percentages=[10, 60, 30],  # capital expenditure distribution, if has_capital_expenditure_period is True
    capital_escalation_percentage=3.6,  # growth rate for capital expenditure costs, default 3.6%
    operating_inflation_percentage=3,  # growth rate for operating costs, default 3%
    revenue_inflation_percentage=3,  # growth rate for revenue, default 3%
    royalty_charge_percentage_of_revenue=6.5,  # percentage of revenue charged as royalties, default 6.5%
    debt_percentage_of_CAPEX=50,  # percentage of capital financed by debt (loans), default 50%
    loan_interest_percentage=6,  # interest rate for loan repayment on capital, default 6%
    loan_repayment_period=10,  # loan repayment period, default 10 years
    capital_depreciation_declining_balance_percentage=150,  # declining balance depreciation rate for capital depreciation, default 150%
)

By default, royalties are calculated as a percentage of the total sales revenue using the value set by `roaylty_charge_percentage_of_revenue`. If desired, users may set an argument `royalty_expression` as an `Expression` to use instead. Likewise, users may set `debt_expression` to use instead of the `debt_percentage_of_CAPEX`. These are optional arguments and will not be used here.

Now that the costing block has been created with the desired NPV-related arguments, we can set `calculate_NPV` in the `build_process_costs` method to add NPV calculations to the plant costs. Because we did not specify any cost values in the costing block, the NPV method will look in the main costing block for required capital, operating, and revenue variables.

Let's set up some component rates for reagents and products:

In [18]:
# create a resource for variable costs
m.fs.chemicals = Var([0], initialize=25, units=pyunits.gallon / pyunits.hr)
m.fs.chemicals.fix()

# define product rates to calculat the revenue and related operating costs
pure_product_output_rates = {
    "Sc2O3": 10 * pyunits.kg / pyunits.day,
    "Dy2O3": 1 * pyunits.kg / pyunits.day,
    "Gd2O3": 1 * pyunits.kg / pyunits.day,
}
mixed_product_output_rates = {
    "Sc2O3": 0.01 * pyunits.kg / pyunits.day,
    "Y2O3": 0.05 * pyunits.kg / pyunits.day,
    "La2O3": 0.05 * pyunits.kg / pyunits.day,
    "CeO2": 0.5 * pyunits.kg / pyunits.day,
}

Now, let's build the plant costs. To reduce the code required to set up the model, we will set the total purchase cost as a value. Note that this value is the total bare erected cost of the plant; the total capital cost will be calculated as the sum of the bare erected costs, installation costs, and any other equipment-related plant costs (e.g. reagent fills for tanks).

The plant costs are built using the following:

In [19]:
# build process costs with fixed and variable O&M set to True (required to calculate NPV)
# a total purchase cost of 5 million USD and a Lang Factor of 2.97
m.fs.costing.build_process_costs(
    total_purchase_cost=5,
    Lang_factor=2.97,
    fixed_OM=True,
    pure_product_output_rates=pure_product_output_rates,
    mixed_product_output_rates=mixed_product_output_rates,
    variable_OM=True,
    resources=["chemicals",],
    rates=[m.fs.chemicals],
    prices={"chemicals": 1.00 * pyunits.USD_2021 / pyunits.gallon},
    calculate_NPV=True,
)

# solve and display results
solver.solve(m, tee=True)
m.fs.costing.display()

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes fo

We've added several new variables to the model; present values are assumed to be negative unless otherwise noted. The attribute `REVENUE` is a `Reference` to `total_sales_revenue` and it is required because Pyomo bars assigning an object to two places in the same model.

The NPV method solves for the following variables:

* `pv_capital_cost` - the present value of all capital costs over the plant lifetime
* `pv_operating_cost` - the present value of all operating costs over the plant lifetime
* `pv_revenue_cost` - the present value of all revenue over the plant lifetime (positive value)
* `pv_royalties_cost` - the present value of all royalties over the plant lifetime
* `loan_debt` - principal loan taken out on capital
* `loan_annual_payment` - yearly payment used to calculate loan interest costs
* `pv_loan_interest` - the present value of all loan interest payments over the plant lifetime
* `pv_capital_depreciation` - the present value of capital depreciation over the plant lifetime
* `npv` - the net present value of the entire plant over its lifetime

and solves a new constraint corresponding to each new variable.

Let's look at the costing report:

In [20]:
m.fs.costing.report()


costing
------------------------------------------------------------------------------------
                                                                                      Value 
    Total Plant Cost [$MM]                                                            14.850
    Total Bare Erected Cost [$MM]                                                     5.0000
    Total Installation Cost [$MM]                                                     9.8500
    Total Other Plant Costs [$MM/year]                                                0.0000
    Total Fixed Operating & Maintenance Cost [$MM/year]                               5.6915
    Total Annual Operating Labor Cost [$MM/year]                                      3.0730
    Total Annual Technical Labor Cost [$MM/year]                                      1.1801
    Summation of Annual Labor Costs [$MM/year]                                        4.2531
    Total Maintenance and Material Cost [$MM/year]                   

We see that net present value is positive, which means that the plant is economically viable - revenue streams outweigh cost streams over the plant lifetime.

# 4.2 Calculate NPV From Fixed Inputs

The NPV calculation method may be utilized without calling the rest of the costing framework by setting fixed inputs for cost components when instantiating the costing block. This approach does not require any knowledge about the flowsheet, resources or product rates.

To begin, build the flowsheet costing block and specify the required cost components.

To demonstrate that the two approaches are equivalent, we will use the values from the results above:

In [21]:
print(value(m.fs.costing.CAPEX))
print(value(m.fs.costing.OPEX))
print(value(m.fs.costing.REVENUE[None]))  # REVENUE is a Reference, and therefore an indexed variable

14.850000000000001
7.031398311359999
14.20951056


The total capital cost is 14.850 million USD, the total operating cost is the sum of the fixed and variable operating costs (7.0314 million USD), and the total sales revenue is 14.210 million USD. The three arguments are required, and if one is set then all three must be set:

In [22]:
# Create a Concrete Model as the top level object
m = ConcreteModel()

# Add a flowsheet object to the model
m.fs = FlowsheetBlock(dynamic=False)

# attach a flowsheet costing block with configuration arguments for NPV calculation
m.fs.costing = QGESSCosting(
    discount_percentage=10,  # still a required argument, as before
    plant_lifetime=20,  # still a required argument, as before
    # set values for CAPEX, OPEX, and REVENUE
    total_capital_cost=14.850000000000001,  # CAPEX
    annual_operating_cost=7.031398311359999,  # OPEX
    annual_revenue=14.20951056,  # REVENUE
    cost_year="2021",  # tells method to return results in MUSD_2021
    # NPV arguments
    has_capital_expenditure_period=True,  # default is False, so need to set this to True
    # use defaults for all other arguments, don't need to set them here
)

Note the `total_capital_cost` above is the entire capital cost including the bare erected costs, installation costs, and any other plant costs. This is not the same as the `total_purchase_cost` passed to the `build_process_costs` method, which comprises the bare erected costs only.

Users must set the argument `cost_year` to define the currency units. The arguments `total_capital_cost`, `annual_operating_cost`, and `annual_revenue` may be passed as scalars, but can also be passed as `Expression`, `Var`, or `Param` objects with or without units of measurement. If there are units of measurements, the variables will be converted internally to the units set by `cost_year`.

Let's build the plant costs. See below how we don't need to define anything else about the plant, or even set any other arguments, to calculate the NPV:

In [23]:
m.fs.costing.build_process_costs(
    fixed_OM=False,  # the required components don't exist, so we skip the fixed_OM method
    # by default, variable_OM is False
    calculate_NPV=True,
)

# solve and display results
solver.solve(m, tee=True)
m.fs.costing.display()

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes fo

As shown in the output above, only the NPV-related attributes are created to minimize the model size.

Checking the costing block report, we see that we obtain the same NPV as in the prior case. The report method skips the attributes that don't exist, so only the NPV is reported below:

In [24]:
m.fs.costing.report()


costing
------------------------------------------------------------------------------------
                             Value
    Net Present Value [$MM] 10.736



# 5 Costing Bounding

Finally, the costing framework supports calculating costing bounds based on the feedstock capacity and grade of a process. Unlike the NPV method, the costing bounding method is independent of the main costing framework. The bound equations derive from Fritz, A.G., Tarka, T.J. & Mauter, M.S. Assessing the economic viability of unconventional rare earth element feedstocks. Nat Sustain 6, 1103–1112 (2023). https://doi.org/10.1038/s41893-023-01145-1. The study establishes a database of capital and operating expenses for REE recovery and market prices of saleable mineral products, and develops surrogates models to predict the maximum cost threshold of specific process types to determine the economic viability. Thresholds are calculated using a power law:

$ C_{threshold} (2022 USD / kg REE) = \alpha * x^\beta$

where $C_{threshold}$ is the cost threshold, $x$ is a known quantity about the process, and $\alpha$ and $\beta$ are empirical coefficients. A dictionary of coefficients were regressed from process data for a range of process types with $x$ as the total feedstock REE:

$ x = F_{grade}(unitless) * F_{capacity}(tonnes)$

The values [$\alpha$, $\beta$] are available in the public paper and the costing framework code, and are presented below:

* "Total Capital": [81, -0.46]
* "Total Operating": [27, -0.087]
* "Beneficiation": [2.7, -0.15]
* "Beneficiation, Chemical Extraction, Enrichment and Separation": [22, -0.059]
* "Chemical Extraction": [40, -0.46]
* "Chemical Extraction, Enrichment and Separation": [15, -0.19]
* "Enrichment and Separation": [6.7, -0.16]
* "Mining": [25, -0.32]

## 4.1 Quantifying Threshold Bounds

To quantify uncertainty in the threshold estimation, a 95% confidence interval was applied to the values to produce upper and lower bound predictions for the cost threshold per the following equation:

$C_{threshold}^{\pm} (2022 USD / kg REE) = ({\alpha} \pm \alpha_{0.95}) * x^{{\beta} \pm \beta_{0.95}}$

where $C_{threshold}^{-}$ corresponds to the 5% confidence bound and $C_{threshold}^{+}$ corresponds to the 95% confidence bound.

The following values, available in the costing framework code, were produced for [$\alpha$, $\alpha_{0.95}$, $\beta$, $\beta_{0.95}$]:

* "Total Capital": [81, 1.4, -0.46, 0.063]
* "Total Operating": [27, 0.87, -0.087, 0.038]
* "Beneficiation": [2.7, 1.3, -0.15, 0.062]
* "Beneficiation, Chemical Extraction, Enrichment and Separation": [22, 1.28,  -0.059, 0.046]
* "Chemical Extraction, Enrichment and Separation": [15, 15, -0.19, 0.28]
* "Enrichment and Separation": [6.7, 2.8, -0.16, 0.11]
* "Mining": [25, 2.5, -0.32, 0.095]

## 4.2 Usage Example

The method itself is straightforward to implement if the required inputs are provided. Suppose we have a process with a feed input rate of 500 U.S. ton of REE per hour with an REE grade of 356.64 ppm.

In [25]:
# Create a Concrete Model as the top level object
m = ConcreteModel()

# Add a flowsheet object to the model
m.fs = FlowsheetBlock(dynamic=False)

# attach a flowsheet costing block with configuration arguments for NPV calculation
m.fs.costing = QGESSCosting()

# define inputs
m.fs.feed_input = Var(initialize=500, units=pyunits.ton / pyunits.hr)
m.fs.feed_grade = Var(initialize=356.64, units=pyunits.ppm)

# define annual operating hours
hours_per_shift = 8
shifts_per_day = 3
operating_days_per_year = 336

m.fs.annual_operating_hours = Param(
    initialize=hours_per_shift * shifts_per_day * operating_days_per_year,
    mutable=True,
    units=pyunits.hours / pyunits.a,  # pyunits.a is "annum", equivalent to pyunits.year
)

# define plant lifetime
m.fs.plant_lifetime = Param(
    initialize=20,
    mutable=True,
    units=pyunits.a
)

# call costing bound method
CE_index_year = "2021"
QGESSCostingData.calculate_REE_costing_bounds(
    b=m.fs.costing,  # block to attach costing bound variables and constraints to
    capacity=m.fs.feed_input * m.fs.annual_operating_hours * m.fs.plant_lifetime,  # lifetime capacity
    grade=m.fs.feed_grade,  # feed grade
    CE_index_year=CE_index_year,  # project dollar year
)

2024-07-30 10:39:59 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New variable 'capacity' created as attribute of fs.costing
2024-07-30 10:39:59 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New variable 'grade' created as attribute of fs.costing
2024-07-30 10:39:59 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New variable 'costing_lower_bound' created as attribute of fs.costing
2024-07-30 10:39:59 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New variable 'costing_upper_bound' created as attribute of fs.costing
2024-07-30 10:39:59 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New constraint 'costing_lower_bounding_eq' created as attribute of fs.costing
2024-07-30 10:39:59 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: New constraint 'costing_upper_bounding_eq' created as attribute of fs.costing
2024-07-30 10:39:59 [INFO] idaes.prommis.uky.costing.ree_plant_capcost: 

Printing calculated costing bounds for processes:
Total Capital : [ 0.3384067822001

The results above were calculated directly from the inputs, and in this case are the correct values. This is not the case for flowsheets in general, as we did not solve the flowsheet yet.

Solving, we confirm the same results:

In [26]:
# solve and display results
solver.solve(m, tee=True)

processes = [
    "Total Capital",
    "Total Operating",
    "Beneficiation",
    "Beneficiation, Chemical Extraction, Enrichment and Separation",
    "Chemical Extraction",
    "Chemical Extraction, Enrichment and Separation",
    "Enrichment and Separation",
    "Mining",
]

print("\n\nPrinting calculated costing bounds for processes:")
for p in processes:
    print(
        p,
        ": [",
        value(m.fs.costing.costing_lower_bound[p]),
        ", ",
        value(m.fs.costing.costing_upper_bound[p]),
        "]",
        getattr(pyunits, "USD_" + CE_index_year),
        "/kg",
    )

Ipopt 3.13.2: nlp_scaling_method=gradient-based
tol=1e-06
max_iter=200


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes fo

# Summary

The advanced method demonstrated here expand upon the capabilities of the REE Costing Framework. Syntax for the example WaterTAP and custom models may be applied to any imported WaterTAP or custom model, respectively, and it is recommended that additional unit operation specificaitions are placed prior to calling the `build_process_costs`. The NPV calculations must be set up when instantiating the flowsheet costing block itself, as shown in this notebook, while the costing bounding equations are entirely independent of the capital and operating cost equations and may be added at any point while building the flowsheet.